In [12]:
import io
import json
import boto3
import pandas as  pd

session = boto3.Session(profile_name='AWS_Profile')

In [14]:
def S3_read(bucket, filename):
    s3 = session.client('s3')
    obj = s3.get_object(Bucket=bucket, Key=filename)
    
    return pandas.read_csv(io.BytesIO(obj['Body'].read()), header=None, sep=';')

def db_put_item(dict_info):
    # dynamo DB 
    dynamodb = session.resource('dynamodb')
    table = dynamodb.Table('customers-accesstoken')
    
    if isinstance(dict_info['itemidwebhook'], str):
        table.put_item(
            Item = dict_info
        )
        return True
    return False

In [15]:
bucket = 'bucket_name'
object_key = 'folder/file.csv'
    
# reading file from S3
df = S3_read(bucket, object_key)
df.head()

# saving rows in dynamo DB
df.apply(lambda x:
    db_put_item(
        {
            'itemidwebhook': x[4],
            'accesstoken': x[3],
            'customersid': str(x[1])
        }
    ), axis=1
)

0        True
1        True
2        True
3        True
4        True
         ... 
15832    True
15833    True
15834    True
15835    True
15836    True
Length: 15837, dtype: bool